In [16]:
import tensorflow as tf
# tensorflow_io provides extended data formats and
# the ability to stream data
import tensorflow_io as tfio
import os
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Audio
# Import AudioSegment from pydub
from pydub import AudioSegment
from pydub.playback import play as playAudio
import keyboard


## Establish our Setup Parameters

In [19]:
file_paths = [os.getcwd() + '/tronica_trimmed.wav', os.getcwd() + '/2023_01_11_rocktronica_AE.wav']
sample_rate = 48000
# in number of seconds
slice_length = 10
# number of samples we will use for our slices
s_len = sample_rate*slice_length
print("samples in slice: ", s_len)
audio_segments = []

# load the file and play it as a sanity check...
ittr = 0
for path in file_paths:
      audio_segments.append(AudioSegment.from_wav(path))
      playAudio(audio_segments[ittr])
      ittr = ittr + 1

samples in slice:  480000


## Generally we want to work with mono audio
 Below code will take the sample and split it into two Mono samples

In [ ]:

  
# Open the stereo audio file as
# an AudioSegment instance
"""
def loadMonoAudioIoTensorFromeStereoFile(path):
    stereo_audio = AudioSegment.from_file(
        path,
        format="wav")
    # read the audio file into an IOTensor
    # stereo_io_tensor = tfio.audio.AudioIOTensor(contents)
    # slice the audio tensor into a bunch of tensors that are each 10 seconds long
    mono_audio = stereo_audio.split_to_mono()
    print(type(mono_audio), len(mono_audio))
    print(mono_audio)
    return mono_audio

mono_audio = loadMonoAudioIoTensorFromeStereoFile(file_path)
"""


In [22]:
# instead lets both load, split, and save stereo files as mono =)
def createMonoFilesFromStereo(path):
    stereo_audio = AudioSegment.from_file(
            path,
            format="wav")
    # Calling the split_to_mono method
    # on the stereo audio file
    mono_audios = stereo_audio.split_to_mono()
    
    # Exporting/Saving the two mono
    # audio files present at index 0(left)
    # and index 1(right) of list returned
    # by split_to_mono method
    left_path = path[:-4]+"_left.wav"
    right_path = path[:-4]+"_right.wav"
    print(left_path)
    print(right_path)
    mono_left = mono_audios[0].export(
        left_path,
        format="wav")
    mono_right = mono_audios[1].export(
        right_path,
        format="wav")
    return mono_left, mono_right

# run the function on our stereo file, and save the two mono audio io tensors 
left_channel, right_channel = createMonoFilesFromStereo(file_paths[1])
print(type(left_channel), left_channel)

/Users/nathan/workspace/tensor_flow_projects/audio_experiments/2023_01_11_rocktronica_AE_left.wav
/Users/nathan/workspace/tensor_flow_projects/audio_experiments/2023_01_11_rocktronica_AE_right.wav
<class '_io.BufferedRandom'> <_io.BufferedRandom name='/Users/nathan/workspace/tensor_flow_projects/audio_experiments/2023_01_11_rocktronica_AE_left.wav'>


In [31]:
audio_tensors = []
audio_tensors.append(tfio.audio.AudioIOTensor(file_paths[0]))
print(type(audio_tensors[0]))
slice_tensor = tf.cast(audio_tensors[0], tf.float32) / 32768.0
print(slice_tensor)
# add a fade out at the start and end
faded_tensor = tfio.audio.fade(slice_tensor, fade_in=1000, fade_out=1000, mode="logarithmic")
plt.figure()
plt.plot(faded_tensor.numpy())

<class 'tensorflow_io.python.ops.audio_ops.AudioIOTensor'>


ValueError: TypeError: Scalar tensor has no `len()`
Traceback (most recent call last):

  File "/Users/nathan/workspace/tensor_flow_projects/tensor_venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py", line 1102, in __len__
    raise TypeError("Scalar tensor has no `len()`")

TypeError: Scalar tensor has no `len()`



Now we are going to create a spectrogram of our audio file...

In [ ]:
fft_size = 512
window_size = 512
stride = 256

spectrogram = tfio.audio.spectrogram(
    faded_tensor, nfft=fft_size, window=window_size, stride=stride)
print(spectrogram.shape)
# remove an extra single dimension axis,
# I am not sure where it comes from...
squeezed_spectrogram = tf.squeeze(tf.math.log(spectrogram).numpy(), axis=[-2])
print(squeezed_spectrogram.shape)

In [ ]:
print(spectrogram)

Now lets plot our spectrogram

In [ ]:
# derr there is something wrong with how this displays....
plt.figure()
plt.imshow(squeezed_spectrogram)
# plt.specgram(spectrogram[0], NFFT=fft_size, window=window_size)

In [ ]:
Audio(faded_tensor[:1].numpy(), rate=audio.rate.numpy())

In [ ]:
#Playback the audio file

# save the tensor as a string that can be saved as a .wav file
# unable to encode as the tensor is an int tensor and not a float tensor
tf.audio.encode_wav(
    audio_slice, sample_rate, name="test.wav"
)

In [ ]:
t = np.arange(0, audio.len(), 1)
fig, (ax1, ax2) = plt.subplots(nrows=2)
plt.plot(audio)
plt.spectrogram(audio)
plt.show()

In [ ]:
plt.spectrogram(audio);

In [ ]:
tf.autograph.to_graph(contents)

In [ ]:
audio2 = tf.audio.decode_wav(
    contents, desired_channels=2
)
print(audio2)